### Movie Dataset

In [82]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = r"C:\Users\John.Mathis\Documents\School\imdb_top_1000.csv"
movies_df = pd.read_csv(file_path)


In [71]:
# Print first few rows of the dataset
movies_df.head()

,Unnamed: 0,Title,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
0,0,Avatar,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000,77025481,785221649,2138484377,2923706026,16-Dec-09,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
1,1,Avengers: Endgame,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000,357115007,858373000,1941066100,2799439100,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
2,2,Avatar: The Way of Water,Jake Sully lives with his newfound family form...,2022,20th Century Studios,December 14 2022 (EMEA APAC),134100226,684075767,1636174514,2320250281,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
3,3,Titanic,A seventeen-year-old aristocrat falls in love ...,1997,Paramount Pictures,200000000,28638131,674292608,1590450697,2264743305,19-Dec-97,"['Drama', 'Romance']",3 hr 14 min,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,"As a new threat to the galaxy rises, Rey, a de...",2015,Walt Disney Studios Motion Pictures,245000000,247966675,936662225,1134647993,2071310218,16-Dec-15,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13


In [72]:
# Get general information about the dataframe
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  1000 non-null   int64 
 1   Title                       1000 non-null   object
 2   Movie Info                  1000 non-null   object
 3   Year                        1000 non-null   int64 
 4   Distributor                 999 non-null    object
 5   Budget (in $)               1000 non-null   object
 6   Domestic Opening (in $)     1000 non-null   object
 7   Domestic Sales (in $)       1000 non-null   int64 
 8   International Sales (in $)  1000 non-null   int64 
 9   World Wide Sales (in $)     1000 non-null   int64 
 10  Release Date                1000 non-null   object
 11  Genre                       1000 non-null   object
 12  Running Time                1000 non-null   object
 13  License                     943 non-null    objec

In [73]:
# Summary statistics of numerical columns
movies_df.describe()

,Unnamed: 0,Year,Domestic Sales (in $),International Sales (in $),World Wide Sales (in $)
count,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03
mean,499.500000,2008.181000,1.646405e+08,2.640890e+08,4.228430e+08
std,288.819436,10.585854,1.197541e+08,2.133847e+08,3.054844e+08
min,0.000000,1937.000000,6.752000e+03,2.450000e+07,1.799689e+08
25%,249.750000,2002.000000,9.572506e+07,1.321190e+08,2.272899e+08
50%,499.500000,2010.000000,1.349169e+08,1.941077e+08,3.197148e+08
75%,749.250000,2016.000000,1.983993e+08,3.188993e+08,4.852363e+08
max,999.000000,2023.000000,9.366622e+08,2.138484e+09,2.923706e+09


In [74]:
movies_df = pd.read_csv(file_path, index_col='Title') # Change index to be Title column
movies_df.head(4)

,Unnamed: 0,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
Title,,,,,,,,,,,,,
Avatar,0,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000,77025481,785221649,2138484377,2923706026,16-Dec-09,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
Avengers: Endgame,1,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000,357115007,858373000,1941066100,2799439100,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
Avatar: The Way of Water,2,Jake Sully lives with his newfound family form...,2022,20th Century Studios,December 14 2022 (EMEA APAC),134100226,684075767,1636174514,2320250281,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
Titanic,3,A seventeen-year-old aristocrat falls in love ...,1997,Paramount Pictures,200000000,28638131,674292608,1590450697,2264743305,19-Dec-97,"['Drama', 'Romance']",3 hr 14 min,PG-13


In [75]:
movies_df.tail(10)

,Unnamed: 0,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
Title,,,,,,,,,,,,,
Scooby-Doo 2: Monsters Unleashed,990,The Mystery Inc. gang must save Coolsville fro...,2004,Warner Bros.,March 26 2004 (10 markets),29438331,84239132,97000000,181239132,10-May-06,"['Action', 'Adventure', 'Thriller']",1 hr 38 min,PG-13
Hot Shots!,991,A parody of Top Gun in which a talented but un...,1991,Twentieth Century Fox,26000000,10848182,69467617,111628547,181096164,31-Jul-91,"['Action', 'Comedy']",1 hr 24 min,NaN
Road to Perdition,992,A mob enforcer's son in 1930s Illinois witness...,2002,DreamWorks Distribution,80000000,22079481,104454762,76546716,181001478,12-Jul-02,"['Crime', 'Drama', 'Thriller']",1 hr 57 min,R
Kill Bill: Vol. 1,993,"After awakening from a four-year coma, a forme...",2003,Miramax,30000000,22200000,70099045,110807031,180906076,9-Oct-03,"['Action', 'Crime', 'Thriller']",1 hr 51 min,R
The Scorpion King,994,A desert warrior rises up against the evil arm...,2002,Universal Pictures,60000000,36075875,91047077,89583830,180630907,18-Apr-02,"['Action', 'Adventure', 'Fantasy']",1 hr 32 min,PG-13
Sweet Home Alabama,995,A young woman who has reinvented herself as a ...,2002,Walt Disney Studios Motion Pictures,30000000,35648740,127223418,53399006,180622424,27-Sep-02,"['Comedy', 'Romance']",1 hr 48 min,PG-13
Daddy's Home 2,996,Having finally gotten used to each other's exi...,2017,Paramount Pictures,69000000,29651193,104029443,76584381,180613824,9-Nov-17,['Comedy'],1 hr 40 min,PG-13
Hacksaw Ridge,997,World War II American Army Medic Desmond T. Do...,2016,Lionsgate,40000000,15190758,67209615,113354021,180563636,3-Nov-16,"['Biography', 'Drama', 'History', 'War']",2 hr 19 min,R
Deja Vu,998,"After a ferry is bombed in New Orleans, an A.T...",2006,Walt Disney Studios Motion Pictures,75000000,20574802,64038616,116518934,180557550,22-Nov-06,"['Action', 'Crime', 'Sci-Fi', 'Thriller']",2 hr 6 min,PG-13


In [76]:
movies_df.rename(columns={'Unnamed: 0': 'Rank'}, inplace=True) # Changing column name
# Update ranking to start at 1 on first movie
if np.issubdtype(movies_df['Rank'].dtype, np.number):
    movies_df.loc[movies_df['Rank'].notnull(), 'Rank'] += 1
movies_df

,Rank,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
Title,,,,,,,,,,,,,
Avatar,1,A paraplegic Marine dispatched to the moon Pan...,2009,Twentieth Century Fox,237000000,77025481,785221649,2138484377,2923706026,16-Dec-09,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
Avengers: Endgame,2,After the devastating events of Avengers: Infi...,2019,Walt Disney Studios Motion Pictures,356000000,357115007,858373000,1941066100,2799439100,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
Avatar: The Way of Water,3,Jake Sully lives with his newfound family form...,2022,20th Century Studios,December 14 2022 (EMEA APAC),134100226,684075767,1636174514,2320250281,24-Apr-19,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
Titanic,4,A seventeen-year-old aristocrat falls in love ...,1997,Paramount Pictures,200000000,28638131,674292608,1590450697,2264743305,19-Dec-97,"['Drama', 'Romance']",3 hr 14 min,PG-13
Star Wars: Episode VII - The Force Awakens,5,"As a new threat to the galaxy rises, Rey, a de...",2015,Walt Disney Studios Motion Pictures,245000000,247966675,936662225,1134647993,2071310218,16-Dec-15,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sweet Home Alabama,996,A young woman who has reinvented herself as a ...,2002,Walt Disney Studios Motion Pictures,30000000,35648740,127223418,53399006,180622424,27-Sep-02,"['Comedy', 'Romance']",1 hr 48 min,PG-13
Daddy's Home 2,997,Having finally gotten used to each other's exi...,2017,Paramount Pictures,69000000,29651193,104029443,76584381,180613824,9-Nov-17,['Comedy'],1 hr 40 min,PG-13
Hacksaw Ridge,998,World War II American Army Medic Desmond T. Do...,2016,Lionsgate,40000000,15190758,67209615,113354021,180563636,3-Nov-16,"['Biography', 'Drama', 'History', 'War']",2 hr 19 min,R


In [77]:
movies_df.shape

(1000, 13)

In [78]:
movies_df.size

13000

In [80]:
movies_df.iloc[199] # Get info about movie rank 200

Rank                                                                        200
Movie Info                    Star race car Lightning McQueen and his pal Ma...
Year                                                                       2011
Distributor                                 Walt Disney Studios Motion Pictures
Budget (in $)                                                         200000000
Domestic Opening (in $)                                                66135507
Domestic Sales (in $)                                                 191452396
International Sales (in $)                                            368400000
World Wide Sales (in $)                                               559852396
Release Date                                                          22-Jun-11
Genre                         ['Adventure', 'Animation', 'Comedy', 'Crime', ...
Running Time                                                        1 hr 46 min
License                                 

In [ ]:
movies_df.loc['']

In [81]:
movies_df.sort_index().head()

,Rank,Movie Info,Year,Distributor,Budget (in $),Domestic Opening (in $),Domestic Sales (in $),International Sales (in $),World Wide Sales (in $),Release Date,Genre,Running Time,License
Title,,,,,,,,,,,,,
"10,000 BC",610,"In the prehistoric past, D'Leh is a mammoth hu...",2008,Warner Bros.,105000000,35867488,94784201,175000000,269784201,5-Mar-08,"['Action', 'Adventure', 'Drama', 'Fantasy', 'H...",1 hr 49 min,PG-13
101 Dalmatians,497,An evil high-fashion designer plots to steal d...,1996,Walt Disney Studios Motion Pictures,November 27 1996 (Domestic),33504025,136189294,184500000,320689294,19-Aug-09,"['Adventure', 'Drama', 'War']",2 hr 33 min,R
102 Dalmatians,973,Cruella DeVil gets out of prison and goes afte...,2000,Walt Disney Studios Motion Pictures,85000000,19883351,66957026,116654745,183611771,22-Nov-00,"['Adventure', 'Comedy', 'Family']",1 hr 40 min,NaN
12 Years a Slave,948,"In the antebellum United States, Solomon North...",2013,Fox Searchlight,20000000,923715,56671993,131061209,187733202,18-Oct-13,"['Biography', 'Drama', 'History']",2 hr 14 min,R
1917,366,"April 6th, 1917. As an infantry battalion asse...",2019,Universal Pictures,95000000,576216,159227644,225351828,384579472,25-Dec-19,"['Action', 'Drama', 'War']",1 hr 59 min,R
